In [1]:
#Example XGBRegression
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import scipy
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from lineartree import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import metrics
import matplotlib.pyplot as plt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [3]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [4]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [5]:
def Tree_Reg(space):    
    #Test Linear Regression on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized
    # load dataset into Pandas DataFrame (cleaned UV data)
    df = pd.read_csv('CdSe PL Train Val.csv')

    #normalize input and output data
    min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
    df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

    X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
    X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
    y = df['Peak_eV']
    z = df['FWHM_eV']
    group = df['Expt. No']

    regressor_peak = DecisionTreeRegressor(max_depth=10, 
                                           min_samples_split = 10, 
                                           min_samples_leaf = 6, 
                                           max_features = 5,
                                              random_state=0)
    regressor_fwhm = DecisionTreeRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                              random_state=0)
    regressor_cascade = DecisionTreeRegressor(max_depth=int(space['x1']), 
                                           min_samples_split = int(space['x2']), 
                                           min_samples_leaf = int(space['x3']), 
                                           max_features = int(space['x4']),
                                              random_state=0)

    gkf = GroupKFold(n_splits=10)
    matrix = np.zeros([10,13])
    matrix_fwhm = np.zeros([10,13])
    matrix_cascade = np.zeros([10,14])
    for i in range(0,1):
        j = 0
        for train_index, test_index in gkf.split(X, groups=group):
            #Separate out training and test sets
            X_train = X.iloc[train_index, :]
            X2_train = X2.iloc[train_index, :]
            y_train = y[train_index]
            z_train = z[train_index]
            X_test = X.iloc[test_index, :]
            y_test = y[test_index]
            z_test = z[test_index]

            #prediction for Peak
            regressor_peak.fit(X_train, y_train)
            y_pred = regressor_peak.predict(X_test)
            #prediction for FWHM (direct)
            regressor_fwhm.fit(X_train, z_train)
            y_pred_fwhm = regressor_fwhm.predict(X_test)
            #prediction for FWHM (cascade)
            regressor_cascade.fit(X2_train, z_train)
            X_test.insert(5, "Peak_eV", regressor_peak.predict(X_test), True)
            y_pred_cascade = regressor_cascade.predict(X_test)

            #errors in eV
            matrix[10*i + j,0] = metrics.mean_absolute_error(y_test, y_pred)
            matrix[10*i + j,1] = metrics.r2_score(y_test, y_pred)
            matrix[10*i + j,2] = rae(y_test, y_pred)
            matrix[10*i + j,3] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

            matrix_fwhm[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,1] = metrics.r2_score(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,2] = rae(z_test, y_pred_fwhm)
            matrix_fwhm[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))

            matrix_cascade[10*i + j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,1] = metrics.r2_score(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,2] = rae(z_test, y_pred_cascade)
            matrix_cascade[10*i + j,3] = np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))

            #errors in nm
            #UNnormalize the data first
            y_test = y_test*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            y_pred = y_pred*(1239.84193/420 - 1239.84193/620)+1239.84193/620
            z_test = z_test*(0.35)+0.05
            y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
            y_pred_cascade = y_pred_cascade*(0.35)+0.05
            matrix[10*i + j,4] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,5] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,6] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
            matrix[10*i + j,7] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
            matrix[10*i + j,8:13] = regressor_peak.feature_importances_

            matrix_fwhm[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
            matrix_fwhm[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
            matrix_fwhm[10*i + j,8:13] = regressor_fwhm.feature_importances_

            matrix_cascade[10*i + j,4] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,5] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,6] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
            matrix_cascade[10*i + j,7] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
            matrix_cascade[10*i + j,8:14] = regressor_cascade.feature_importances_

            j+=1

    matrix_2 = np.zeros([1,21])
    matrix_2[0,:8] = matrix.mean(0)[:8]
    matrix_2[0,8:16] = np.std(matrix[:,:8], axis=0)
    matrix_2[0,16:21] = matrix.mean(0)[8:13]

    matrix_3 = np.zeros([1,21])
    matrix_3[0,:8] = matrix_fwhm.mean(0)[:8]
    matrix_3[0,8:16] = np.std(matrix_fwhm[:,:8], axis=0)
    matrix_3[0,16:21] = matrix_fwhm.mean(0)[8:13]

    matrix_4 = np.zeros([1,22])
    matrix_4[0,:8] = matrix_cascade.mean(0)[:8]
    matrix_4[0,8:16] = np.std(matrix_cascade[:,:8], axis=0)
    matrix_4[0,16:22] = matrix_cascade.mean(0)[8:14]

    print('\033[1m' + 'Absorption Peak' + '\033[0m')
    print("MAE (norm, eV):", matrix_2[0,0], matrix_2[0,8])
    print("R2 (norm, eV):", matrix_2[0,1], matrix_2[0,9])
    print("RAE (norm, eV):", matrix_2[0,2], matrix_2[0,10])
    print("RMSE (norm, eV):", matrix_2[0,3], matrix_2[0,11])
    print("Importances", matrix_2[0,8:13])
    print("MAE (nm):", matrix_2[0,4], matrix_2[0,12])
    print("R2 (nm):", matrix_2[0,5], matrix_2[0,13])
    print("RAE (nm):", matrix_2[0,6], matrix_2[0,14])
    print("RMSE (nm):", matrix_2[0,7], matrix_2[0,15])

    print('\033[1m' + 'Absorption FWHM (direct)' + '\033[0m')
    print("MAE (norm, eV):", matrix_3[0,0], matrix_3[0,8])
    print("R2 (norm, eV):", matrix_3[0,1], matrix_3[0,9])
    print("RAE (norm, eV):", matrix_3[0,2], matrix_3[0,10])
    print("RMSE (norm, eV):", matrix_3[0,3], matrix_3[0,11])
    print("Importances", matrix_3[0,8:13])
    print("MAE (nm):", matrix_3[0,4], matrix_3[0,12])
    print("R2 (nm):", matrix_3[0,5], matrix_3[0,13])
    print("RAE (nm):", matrix_3[0,6], matrix_3[0,14])
    print("RMSE (nm):", matrix_3[0,7], matrix_3[0,15])

    print('\033[1m' + 'Absorption FWHM (cascade)' + '\033[0m')
    print("MAE (norm, eV):", matrix_4[0,0], matrix_4[0,8])
    print("R2 (norm, eV):", matrix_4[0,1], matrix_4[0,9])
    print("RAE (norm, eV):", matrix_4[0,2], matrix_4[0,10])
    print("RMSE (norm, eV):", matrix_4[0,3], matrix_4[0,11])
    print("Importances", matrix_4[0,8:14])
    print("MAE (nm):", matrix_4[0,4], matrix_4[0,12])
    print("R2 (nm):", matrix_4[0,5], matrix_4[0,13])
    print("RAE (nm):", matrix_4[0,6], matrix_4[0,14])
    print("RMSE (nm):", matrix_4[0,7], matrix_4[0,15])

    return {'loss': matrix_4[0,3] ** 2, 'status': STATUS_OK }

In [6]:
rstate = np.random.default_rng(42)

from hyperopt import fmin, tpe, hp, STATUS_OK

space = { 'x1': hp.quniform('x1', 10,20,1),
            'x2': hp.quniform('x2', 2,20,1),
            'x3': hp.quniform('x3', 1,10,1),
            'x4': hp.quniform('x4', 1,5,1),
        }

best = fmin(Tree_Reg,
            space=space,
            algo=tpe.suggest,
            max_evals=200,
            rstate=rstate)

print(best)

Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.05353954121328162                                                                                                    
0.0016715798262959356                                                                                                  
R2 (norm, eV):                                                                                                         
0.7867501303279244                                                                                                     
0.02227377415140765                                                                                                    
RAE (norm, eV):                                                                                                        
0.4411682536793437                              

[0.00199093 0.06742104 0.03432119 0.0029667  0.15636227 0.09446833]                                                    
MAE (nm):                                                                                                              
2.9389553220935687                                                                                                     
0.15636227459628854                                                                                                    
R2 (nm):                                                                                                               
0.5048528471081858                                                                                                     
0.09446832925696647                                                                                                    
RAE (nm):                                                                                                              
0.6420757964888567                      

MAE (norm, eV):                                                                                                        
0.030542756697954383                                                                                                   
0.0027175681782602807                                                                                                  
R2 (norm, eV):                                                                                                         
0.4604962553439783                                                                                                     
0.10262424819601922                                                                                                    
RAE (norm, eV):                                                                                                        
0.6595906015224242                                                                                                     
0.06474780577684973                     

MAE (nm):                                                                                                              
3.224453275938461                                                                                                      
0.14672469303533958                                                                                                    
R2 (nm):                                                                                                               
0.4100241635912025                                                                                                     
0.10905698352101541                                                                                                    
RAE (nm):                                                                                                              
0.7048263245017878                                                                                                     
0.05632285186359896                     

0.02990890887559148                                                                                                    
0.003534965892666189                                                                                                   
R2 (norm, eV):                                                                                                         
0.44354337012759243                                                                                                    
0.1399336223965959                                                                                                     
RAE (norm, eV):                                                                                                        
0.6439406763179045                                                                                                     
0.061318164862698796                                                                                                   
RMSE (norm, eV):                        

...

0.06666252007369554                                                                                                    
RMSE (norm, eV):                                                                                                       
0.04302474760879247                                                                                                    
0.00476963708282986                                                                                                    
Importances                                                                                                            
[0.00351143 0.11531362 0.06666252 0.00476964 0.22307892]                                                               
MAE (nm):                                                                                                              
3.0160499316934577                                                                                                     
0.22307892164947626                     

0.02977112831759778                                                                                                    
RMSE (nm):                                                                                                             
15.116515139775169                                                                                                     
0.3904261867525768                                                                                                     
Absorption FWHM (direct)                                                                                       
MAE (norm, eV):                                                                                                        
0.02898759577237553                                                                                                    
0.0038062231411907683                                                                                                  
R2 (norm, eV):                                  

RMSE (norm, eV):                                                                                                       
0.06780490827387833                                                                                                    
0.0024567622098814294                                                                                                  
Importances                                                                                                            
[0.00167158 0.02227377 0.02823382 0.00245676 0.26498889]                                                               
MAE (nm):                                                                                                              
12.020150028416875                                                                                                     
0.2649888864145509                                                                                                     
R2 (nm):                                

RMSE (nm):                                                                                                             
4.213679366183063                                                                                                      
0.19850458125670423                                                                                                    
Absorption Peak                                                                                                
MAE (norm, eV):                                                                                                        
0.05353954121328162                                                                                                    
0.0016715798262959356                                                                                                  
R2 (norm, eV):                                                                                                         
0.7867501303279244                              

0.04154331375412314                                                                                                    
0.003193323458581298                                                                                                   
Importances                                                                                                            
[0.00186046 0.08431786 0.04252767 0.00319332 0.1295249  0.09464374]                                                    
MAE (nm):                                                                                                              
2.9749057255588833                                                                                                     
0.12952490302580746                                                                                                    
R2 (nm):                                                                                                               
0.5004410092679215                      